# OpenAI Tool calling capabilities with Databricks Unity Catalog

Import and run this notebook in Databricks

In [ ]:
%pip install unitycatalog-openai[databricks] mlflow langchain_openai
%restart_python

#### Create a DatabricksFunctionClient

Creating a client that uses a serverless connection endpoint allows for Python function CRUD operations within UC as well as function execution capabilities.

In [0]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

# set the default uc function client, it will be used for all toolkits
set_uc_function_client(client)

#### Create a UC function for executing python code

In [0]:
# replace with your own catalog and schema
CATALOG = "ml"
SCHEMA = "serena_test"


def execute_python_code(code: str) -> str:
    """
    Executes the given python code and returns its stdout.

    Args:
      code: Python code to execute. Remember to print the final result to stdout.
    """
    import sys
    from io import StringIO

    stdout = StringIO()
    sys.stdout = stdout
    exec(code)
    return stdout.getvalue()


function_info = client.create_python_function(
    func=execute_python_code, catalog=CATALOG, schema=SCHEMA, replace=True
)
python_execution_function_name = function_info.full_name

In [0]:
client.execute_function(python_execution_function_name, {"code": "print(1+1)"})

FunctionExecutionResult(error=None, format='SCALAR', value='2\n', truncated=None)

#### Create a UC function for translation

Use Databricks built-in [ai_translate](https://docs.databricks.com/en/sql/language-manual/functions/ai_translate.html) function

In [0]:
translate_function_name = f"{CATALOG}.{SCHEMA}.translate"
sql_body = f"""CREATE OR REPLACE FUNCTION {translate_function_name}(content STRING COMMENT 'the text to be translated', language STRING COMMENT 'the target language code to translate the content to')
RETURNS STRING
RETURN SELECT ai_translate(content, language)
"""
client.create_function(sql_function_body=sql_body)

FunctionInfo(browse_only=None, catalog_name='ml', comment=None, created_at=1729136621857, created_by='serena.ruan@databricks.com', data_type=<ColumnTypeName.STRING: 'STRING'>, external_language=None, external_name=None, full_data_type='STRING', full_name='ml.serena_test.translate', function_id='423d49b1-8c8b-4a14-a6c7-eed1e3a23afc', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='content', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=0, comment='the text to be translated', parameter_default=None, parameter_mode=None, parameter_type=<FunctionParameterType.PARAM: 'PARAM'>, type_interval_type=None, type_json='{"name":"content","type":"string","nullable":true,"metadata":{"comment":"the text to be translated"}}', type_precision=0, type_scale=0), FunctionParameterInfo(name='language', type_text='string', type_name=<ColumnTypeName.STRING: 'STRING'>, position=1, comment='the target language code to translate the content to', parameter_de

In [0]:
client.execute_function(
    translate_function_name, {"content": "What is Databricks?", "language": "es"}
)

FunctionExecutionResult(error=None, format='SCALAR', value='¿Qué es Databricks?', truncated=None)

#### Create tools

In [0]:
from unitycatalog.ai.openai.toolkit import UCFunctionToolkit

toolkit = UCFunctionToolkit(
    function_names=[python_execution_function_name, translate_function_name]
)
tools = toolkit.tools
tools

[{'type': 'function',
  'function': {'name': 'ml__serena_test__execute_python_code',
   'strict': True,
   'parameters': {'properties': {'code': {'anyOf': [{'type': 'string'},
       {'type': 'null'}],
      'description': 'Python code to execute. Remember to print the final result to stdout.',
      'title': 'Code'}},
    'title': 'ml__serena_test__execute_python_code__params',
    'type': 'object',
    'additionalProperties': False,
    'required': ['code']},
   'description': 'Executes the given python code and returns its stdout.'}},
 {'type': 'function',
  'function': {'name': 'ml__serena_test__translate',
   'strict': True,
   'parameters': {'properties': {'content': {'anyOf': [{'type': 'string'},
       {'type': 'null'}],
      'description': 'the text to be translated',
      'title': 'Content'},
     'language': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
      'description': 'the target language code to translate the content to',
      'title': 'Language'}},
    'title'

#### Use the tools in OpenAI

In [0]:
import os

# Replace with your own API key
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

In [0]:
import openai

messages = [
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user.",
    },
    {"role": "user", "content": "What is the result of 2**10?"},
]
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)
# check the model response
response

ChatCompletion(id='chatcmpl-AJBnsZzsh5HVxhGPExplOSwJTwvGs', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_WBDFysLam5QIPaNdD7cpLzHP', function=Function(arguments='{"code":"print(2**10)"}', name='ml__serena_test__execute_python_code'), type='function')]))], created=1729136968, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_e2bde53e6e', usage=CompletionUsage(completion_tokens=26, prompt_tokens=175, total_tokens=201, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))


In [0]:
# Generate the final response using helper functions provided in the sdk
from unitycatalog.ai.openai.utils import generate_tool_call_messages

messages = generate_tool_call_messages(response=response)
messages

[{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_WBDFysLam5QIPaNdD7cpLzHP', 'function': {'arguments': '{"code":"print(2**10)"}', 'name': 'ml__serena_test__execute_python_code'}, 'type': 'function'}]}, {'role': 'tool', 'content': '{"content": "1024\\n"}', 'tool_call_id': 'call_WBDFysLam5QIPaNdD7cpLzHP'}]


In [0]:
# Test the model can choose tools based on question
messages = [
    {
        "role": "system",
        "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user.",
    },
    {"role": "user", "content": "What is English for '我是一个机器人'?"},
]
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)
messages = generate_tool_call_messages(response=response)
messages

[{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_GXwEMdBD2gMvvqjvG1mwgB1f', 'function': {'arguments': '{"content":"我是一个机器人","language":"en"}', 'name': 'ml__serena_test__translate'}, 'type': 'function'}]}, {'role': 'tool', 'content': '{"content": "I am a robot"}', 'tool_call_id': 'call_GXwEMdBD2gMvvqjvG1mwgB1f'}]
